In [86]:
### IMPORTAMOS Y LIMPIAMOS NUESTROS DATASETS

import numpy as np
import pandas as pd

data_idealista_venta = pd.read_csv('./DB_idealista_VENTA_completo.csv', index_col=0)
data_airdna = pd.read_csv('./DB_airDNA_completo.csv', index_col=0)
       

In [87]:

### IDEALISTA



# Separamos en columnas los detalles de los anuncios de venta de idealista

import re

data_idealista_venta['Habitaciones'] = ''
data_idealista_venta['Superficie'] = ''
data_idealista_venta['Planta'] = ''
data_idealista_venta['Exterior'] = ''
data_idealista_venta['Ascensor'] = ''

for i in range(len(data_idealista_venta['Detalles'])):

    texto = data_idealista_venta['Detalles'][i]

    # Extraemos en una columna el dato del nº de habitaciones

    pattern = r"'(\d+) hab.'"
    matchh = re.search(pattern, texto)
    if matchh:
        data_idealista_venta['Habitaciones'][i] = matchh.group(1)

    # Extraemos en una columna el dato de los m2

    pattern = r"'(\d+) m²'"
    matchh = re.search(pattern, texto)
    if matchh:
        data_idealista_venta['Superficie'][i] = matchh.group(1)

    # Extraemos en una columna el dato de la planta en la que se encuentra la vivienda
        
    pattern = r"'Planta (\d+)"
    matchh = re.search(pattern, texto)
    if matchh:
        data_idealista_venta['Planta'][i] = int(matchh.group(1))
    elif 'bajo' in texto.lower():
        data_idealista_venta['Planta'][i] = 0

    # Extraemos en una columna el dato de si es exterior o interior
        
    if "exterior" in texto:
        data_idealista_venta['Exterior'][i] = 1
    elif "interior" in texto:
        data_idealista_venta['Exterior'][i] = 0
    
    # Extraemos en una columna el dato de si tiene o no ascensor
        
    if "con ascensor" in texto:
        data_idealista_venta['Ascensor'][i] = 1
    elif "sin ascensor" in texto:
        data_idealista_venta['Ascensor'][i] = 0

data_idealista_venta.drop(columns = "Detalles", inplace = True)





# Limpiamos un poco los textos y reordenamos las columnas

data_idealista_venta['Título'] = data_idealista_venta['Título'].str.strip()
data_idealista_venta['Descripción'] = data_idealista_venta['Descripción'].str.strip()

data_idealista_venta = data_idealista_venta[['Título', 'Descripción', 'Habitaciones', 'Superficie', 
                                             'Planta', 'Exterior', 'Ascensor', 'Garaje', 'Precio', 'Distrito']]




# Transformamos el tipo de dato de la columna precio, ya que no tenía un formato numerico

data_idealista_venta['Precio'] = data_idealista_venta['Precio'].str.strip('€')
data_idealista_venta['Precio'] = data_idealista_venta['Precio'].str.replace('.','')
data_idealista_venta['Precio'] = data_idealista_venta['Precio'].astype(int)





# Transformamos la columna superficie en int, ya que no tenía valor numérico

data_idealista_venta['Superficie'] = pd.to_numeric(data_idealista_venta['Superficie'], errors='coerce')
data_idealista_venta = data_idealista_venta.dropna(subset=['Superficie'])
data_idealista_venta['Superficie'] = data_idealista_venta['Superficie'].astype(int)





# Agregamos la columna de precio/m2, calculada en base al precio de venta y a la superficie

data_idealista_venta['Precio/m2'] = round(data_idealista_venta['Precio'] / data_idealista_venta['Superficie'], 2)





# Algunos tipos de viviendas (estudios, áticos y dúplex) no tienen ningún valor en nº de habitaciones - lo ponemos en 0

data_idealista_venta['Habitaciones'] = data_idealista_venta.apply(
    lambda x: '0' if x['Habitaciones'] == '' and 
                (('estudio' in x['Título'].lower()) or ('ático' in x['Título'].lower()) or ('dúplex' in x['Título'].lower())) 
                else x['Habitaciones'],
    axis=1
    )




# Si hay algun otro tipo de vivienda donde falta especificar el número de habitaciones lo eliminamos, 
# previamente habiendo comprobado que se trata de un porcentaje muy reducido de nuestro dataset.
# También ajustamos el formato de la columna 'Habitaciones' ya que no tenía formato numérico.


data_idealista_venta = data_idealista_venta[data_idealista_venta['Habitaciones'] != ''] 
data_idealista_venta['Habitaciones'] = data_idealista_venta['Habitaciones'].astype(int)

In [88]:
data_idealista_venta

,Título,Descripción,Habitaciones,Superficie,Planta,Exterior,Ascensor,Garaje,Precio,Distrito,Precio/m2
0,"Piso en calle Nicolás Salmerón, Victoria Eugen...",¡Haz realidad tu propósito de año nuevo! Estre...,2,59,1,1,0,NaN,135000,Bailen-miraflores,2288.14
1,"Piso en Victoria Eugenia, Málaga",Te presentamos este piso que SE ENTREGARIA TOT...,3,81,5,1,1,NaN,199000,Bailen-miraflores,2456.79
2,"Piso en calle Antonio Jiménez Ruiz, Los Castil...",Exclusivo piso ubicado en el barrio de La Trin...,2,62,2,0,1,NaN,175000,Bailen-miraflores,2822.58
3,"Piso en C. Eugenio Gross, 11, Los Castillejos ...","Berkshire Hathaway HomeServices, líder reconoc...",2,104,4,1,1,Garaje incluido,342200,Bailen-miraflores,3290.38
4,"Piso en calle Ecuador, s/n, Los Castillejos - ...",OPORTUNIDAD.\nTe presentamos este encantador p...,3,83,2,1,1,NaN,175000,Bailen-miraflores,2108.43
...,...,...,...,...,...,...,...,...,...,...,...
4680,Chalet adosado en El Tejar - Hacienda Bizcoche...,AYRE ESTATES LE OFRECE EN EXCLUSIVA LA VENTA D...,4,150,,,,NaN,479000,Teatinos,3193.33
4681,Casa o chalet independiente en calle Gaspar de...,"Se vende loft moderno, increíble ubicación, a ...",2,80,,,,NaN,175000,Teatinos,2187.50
4682,"Piso en El Consul - El Romeral, Málaga",International Inmobiliaria te presenta esta pr...,4,125,2,1,1,Garaje incluido,395000,Teatinos,3160.00
4683,"Dúplex en calle Doctor Norman Bethune, 5, El C...","Venta dúplex, semi nuevo en perfecto estado de...",4,100,2,1,1,Garaje incluido,189500,Teatinos,1895.00


In [89]:
### AIRDNA

data_airdna.columns = data_airdna.columns.str.capitalize()
data_airdna.rename(columns={'Bedrooms' : 'Habitaciones',
                            'Bathrooms' : 'Baños',
                            'Accommodates' : 'Guests',
                            'Revenue_ltm' : 'Ganancia (12 meses anteriores) (EUR)',
                            'Revenue_potential_ltm' : 'Ganancia potencial (EUR)',
                            'Occupancy_rate_ltm' : 'Ocupación',
                            'Average_daily_rate_ltm' : 'Tarifa media diaria (EUR)',
                            'Days_available_ltm' : 'Días disponibles (12 meses anteriores)',
                            'Title' : 'Título anuncio',

}, inplace = True)

if ('Market_id' in data_airdna.columns) \
    and ('Market_name' in  data_airdna.columns) \
    and ('Property_id' in  data_airdna.columns) \
    and ('Airbnb_property_id' in  data_airdna.columns) \
    and ('Vrbo_property_id' in  data_airdna.columns):
    data_airdna.drop(columns=['Market_id', 'Market_name', 'Property_id', 'Airbnb_property_id', 'Vrbo_property_id'], inplace=True)

data_airdna = data_airdna[data_airdna['Listing_type'] != 'private_room']
data_airdna.reset_index(drop=True, inplace=True)
data_airdna.drop(columns='Listing_type', inplace=True)

data_airdna = data_airdna[['Título anuncio', 'Habitaciones', 'Baños', 'Guests', 'Rating', 'Reviews',
                           'Ganancia (12 meses anteriores) (EUR)', 'Ganancia potencial (EUR)', 'Ocupación', 'Tarifa media diaria (EUR)', 
                           'Días disponibles (12 meses anteriores)', 'Lat', 'Long', 'Distrito']]

In [90]:
data_airdna

,Título anuncio,Habitaciones,Baños,Guests,Rating,Reviews,Ganancia (12 meses anteriores) (EUR),Ganancia potencial (EUR),Ocupación,Tarifa media diaria (EUR),Días disponibles (12 meses anteriores),Lat,Long,Distrito
0,Apartamento Centro Málaga+Garaje Privado Gratis,1,1.0,4,4.8,810,27010.26,28506.77,84.20,92.18,348,36.722300,-4.427160,Centro
1,Nice Flat in the center VFT/MA/00202,0,1.0,2,4.8,803,17090.18,17684.94,100.00,48.69,351,36.726090,-4.419700,Centro
2,Lovely apartment in the center VFT/MA/00204,1,1.0,2,4.8,720,17183.36,21824.55,100.00,59.26,290,36.724940,-4.420030,Centro
3,"Relax, culture and beach in the heart of the c...",0,1.0,2,5.0,588,22396.48,23691.99,99.42,65.68,343,36.721340,-4.418110,Centro
4,Picasso Home,1,1.0,5,4.8,563,17610.94,24919.90,99.61,69.61,254,36.726770,-4.419330,Centro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,"Beach front Malagueta Miramar, 10 people",4,2.0,10,4.5,68,79611.02,102459.74,83.45,328.97,290,36.717910,-4.408660,Centro
1962,Acogedor apartamento junto al museo Picasso.,1,1.0,2,4.5,68,9415.77,16421.41,83.42,58.48,193,36.721977,-4.418969,Centro
1963,La Mar Salada - MálagadeVacaciones,1,1.0,4,4.6,68,33577.36,39161.41,82.87,126.23,321,36.718110,-4.409590,Centro
1964,Sw Calle Capitan-Charming Duplex,1,1.0,4,4.3,68,21550.47,25030.45,82.62,85.51,305,36.721360,-4.419990,Centro


In [91]:
data_idealista_venta.to_csv('./DB_idealista_VENTA_completo_limpio.csv')
data_airdna.to_csv('./DB_airDNA_completo_limpio.csv')